# Diffie-Hellman Key Exchange Simulator
## Implementing DF Key Exchange to demonstrate secure key exchange using:
    1. Miller-Rabin primality testing for prime verification,
    2. Efficient primitive root discovery using Carmichael's theorem,
    3. Complete protocol simulation with interception testing.

In [1]:
# Prerequisites
import math
import random
from math import gcd
from functools import lru_cache
from typing import List, Set, Tuple
from dataclasses import dataclass

In [2]:
# Constants for cryptographic safety
MIN_DIGITS = 2
MAX_DIGITS = 6
MILLER_RABIN_ROUNDS = 40

In [3]:
"""Creating class to bundle Diffie-Hellman parameters safely"""
@dataclass
class DHParameters:
    """Container for all Diffie-Hellman parameters"""
    prime: int
    primitive_root: int
    ram_private: int = None
    shyam_private: int = None

    def validate(self) -> bool:
        return (miller_rabin(self.prime) and 
               is_primitive_root(self.primitive_root, self.prime))

In [4]:
"""
Implementing the Miller-Rabin primality test with:
- Deterministic checks for numbers < 2^64
- 40 rounds of testing by default
- Optimized witness selection
"""
def miller_rabin(n: int, k: int = MILLER_RABIN_ROUNDS) -> bool:
    if n < 2:
        return False
    for p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]:
        if n % p == 0:
            return n == p

    d, s = n - 1, 0
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for __ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [5]:
"""
Finding prime factors using trial division with:
- Memoization to cache results
- Early termination when possible
- Wheel factorization for Harins
"""
@lru_cache(maxsize=128)
def prime_factors(n: int) -> Set[int]:
    factors = set()
    while n % 2 == 0:
        factors.add(2)
        n //= 2
    i = 3
    while i * i <= n:
        while n % i == 0:
            factors.add(i)
            n //= i
        i += 2
    if n > 2:
        factors.add(n)
    return factors

In [6]:
"""
Checking for primitive roots using:
- Carmichael's theorem optimization
- Precomputed prime factors of φ(p)
- Efficient modular exponentiation
"""
def is_primitive_root(g: int, p: int) -> bool:
    if gcd(g, p) != 1:
        return False
    phi = p - 1
    factors = prime_factors(phi)
    return all(pow(g, phi // f, p) != 1 for f in factors)

In [7]:
"""
Discovering primitive roots by:
- First verifying p is prime
- Checking candidates in smart order
- Using early termination when possible
"""
def find_primitive_roots(p: int) -> List[int]:
    if not miller_rabin(p):
        return []
    
    # Checking small numbers first for efficiency
    candidates = list(range(1, min(p, 100)))
    if p > 100:
        candidates += random.sample(range(100, p), min(100, p-100))
    
    return [g for g in candidates if is_primitive_root(g, p)]

In [8]:
"""Ensure to select a valid primitive root from the list"""
def get_primitive_root(p: int, roots: List[int]) -> int:
    print(f"\nPrimitive roots for {p}:")
    print(roots[:16], "..." if len(roots) > 16 else "")
    while True:
        try:
            choice = int(input(f"Choose primitive root: "))
            if choice in roots:
                return choice
            print(f"Error: {choice} is not a primitive root. Please choose from the list.")
            print("Valid roots:", roots[:16], "...")
        except ValueError:
            print("Invalid input. Please enter a number.")

In [9]:
"""
Safely getting user input by:
- Validating numeric ranges
- Providing clear error messages
- Handling type conversion safely
"""
def get_valid_input(prompt: str, min_val: int, max_val: int) -> int:
    while True:
        try:
            val = int(input(prompt))
            if min_val <= val <= max_val:
                return val
            print(f"Please enter between {min_val}-{max_val}")
        except ValueError:
            print("Invalid number. Please try again.")

In [10]:
"""
Users select primes by:
- Showing sample primes for the digit length
- Allowing custom prime entry
- Validating all inputs thoroughly
"""
def get_prime_candidate(n: int) -> int:
    min_val = 10**(n-1)
    max_val = 10**n - 1
    
    print(f"\nFinding {n}-digit primes...")
    primes = [num for num in range(min_val, max_val + 1) if miller_rabin(num)][:10]
    
    if primes:
        print("\nSample primes:")
        for i, p in enumerate(primes, 1):
            print(f"{i}. {p}")
    
    while True:
        try:
            choice = input(f"\nChoose prime (1-{len(primes)}) or enter custom: ")
            if choice.isdigit() and 1 <= int(choice) <= len(primes):
                return primes[int(choice)-1]
            
            p = int(choice)
            if not (min_val <= p <= max_val):
                print(f"Must be {n}-digit number")
                continue
            if miller_rabin(p):
                return p
            print("Not prime. Try again.")
        except ValueError:
            print("Invalid input. Please try again.")

In [11]:
"""
Executing the core DH protocol by:
- Taking user input for private keys
- Calculating public keys safely
- Verifying shared secret matches
"""
def perform_key_exchange(params: DHParameters) -> Tuple[int, int, int, int]:
    print("\n=== Private Keys ===")
    params.ram_private = get_valid_input(f"Enter Ram's key (1-{params.prime-2}): ", 1, params.prime-2)
    params.shyam_private = get_valid_input(f"Enter Shyam's key (1-{params.prime-2}): ", 1, params.prime-2)
    
    y1 = pow(params.primitive_root, params.ram_private, params.prime)
    y2 = pow(params.primitive_root, params.shyam_private, params.prime)
    
    k1 = pow(y2, params.ram_private, params.prime)
    k2 = pow(y1, params.shyam_private, params.prime)
    
    return y1, y2, k1, k2

In [12]:
"""
Demonstrating Man In The Middle (MITM) vulnerability by:
- Taking attacker's key input
- Showing forged key attempts
- Comparing with real shared secret
"""
def simulate_interception(params: DHParameters, y1: int, y2: int) -> None:
    if params.ram_private is None or params.shyam_private is None:
        raise ValueError("Private keys not initialized")
    
    print("\n=== Interception Attempt ===")
    h = get_valid_input(f"Enter Hari's key (1-{params.prime-2}): ", 1, params.prime-2)
    
    forged_key = pow(params.primitive_root, h, params.prime)   
    print(f"\nHari intercepts and sends forged key, i.e. {forged_key}, to both Ram and Shyam.\n")
    
    # Ram and Shyam compute secrets using the forged key
    ram_compromised_secret = pow(forged_key, params.ram_private, params.prime)
    print(f"Ram computes {ram_compromised_secret} as secret using the forged key.")
    shyam_compromised_secret = pow(forged_key, params.shyam_private, params.prime)
    print(f"Shyam computes {shyam_compromised_secret} as secret using the forged key.")

    # Hari computes the same secrets from intercepted public keys
    intercepted_ram_secret = pow(y1, h, params.prime)
    intercepted_shyam_secret = pow(y2, h, params.prime)
    print(f"\nHari computes {intercepted_ram_secret} as Ram's secret and {intercepted_shyam_secret} as Shyams's secret using the forged key.")

    # Verification
    if ram_compromised_secret == shyam_compromised_secret:
        print("The forged key makes both victims - Ram and Shyam - compute same 'compromised' shared secret.")
        
        if ram_compromised_secret == shyam_compromised_secret == intercepted_ram_secret == intercepted_shyam_secret:
            print("Hari computes the same secret using Ram’s and Shyams’s public keys.")
            print(f"All parties (Ram, Shyam, and Hari) agree on the same compromised secret: {ram_compromised_secret}")
            print("The attack from Hari succeeds, and Ram and Shyam remain unaware of Hari’s presence.")
            print("Then Hari can decrypt and re-encrypt messages and neither Ram nor Shyam may notice the tampering.")
            print("Hence, Interception by Hari is successful!")
        else:
            print("Something is wrong! Please check the Diffie-Hellman Key Exchange Algorithm thoroughly and modify make adjustment to the code accordingly.")
    else:
        print("The forged key leads Ram and Shyam to compute different shared secrets.")
        print("Still neither Ram nor Shyam is aware that they are communicating through Hari.")
        print("The core vulnerability lies in the lack of authentication.")
        print("Without authentication of the parties involved, Hari can easily impersonate one of the parties and establish separate keys with both.")
        print("This allows him to intercept and manipulate and tamper with the communications.")
        print("However, this mismatch in shared secrets still could cause errors in encryption and decryption, making Ram and Shyam suspect of an attack.")
        print("\nHence, Interception by Hari MAY fail!")

In [13]:
"""Orchestrating the complete protocol demonstration"""
def diffie_hellman_exchange():
    print("\n=== Diffie-Hellman Key Exchange ===")
    
    # Getting prime parameters
    n = get_valid_input(f"\nEnter prime digits ({MIN_DIGITS}-{MAX_DIGITS}): ", MIN_DIGITS, MAX_DIGITS)
    p = get_prime_candidate(n)
    
    # Finding primitive roots
    roots = find_primitive_roots(p)
    if not roots:
        print(f"\nNo primitive roots found for {p}!")
        return
    
    g = get_primitive_root(p, roots)
    
    params = DHParameters(p, g)
    
    # Performing key exchange
    y1, y2, k1, k2 = perform_key_exchange(params)
    
    print("\n=== Public Keys ===")
    print(f"Ram's public key: {y1}")
    print(f"Shyam's public key: {y2}")
    
    print("\n=== Shared Secret ===")
    print(f"Ram's secret: {k1}")
    print(f"Shyam's secret: {k2}")
    
    if k1 == k2:
        print("\nKey exchange successful!")
        
        # Get valid y/n input for interception test
        while True:
            test_intercept = input("\nTest interception? (y/n): ").lower().strip()
            if test_intercept == 'y':
                simulate_interception(params, y1, y2)
                break
            elif test_intercept == 'n':
                print("Simulation ended successfully.")
                break
            else:
                print("Please enter only 'y' or 'n'")
    else:
        print("\nKey exchange failed!")

if __name__ == "__main__":
    diffie_hellman_exchange()


=== Diffie-Hellman Key Exchange ===



Enter prime digits (2-6):  2



Finding 2-digit primes...

Sample primes:
1. 11
2. 13
3. 17
4. 19
5. 23
6. 29
7. 31
8. 37
9. 41
10. 43



Choose prime (1-10) or enter custom:  5



Primitive roots for 23:
[5, 7, 10, 11, 14, 15, 17, 19, 20, 21] 


...


=== Public Keys ===
Ram's public key: 4
Shyam's public key: 10

=== Shared Secret ===
Ram's secret: 18
Shyam's secret: 18

Key exchange successful!



Test interception? (y/n):  y



=== Interception Attempt ===


Enter Hari's key (1-21):  3



Hari intercepts and sends forged key, i.e. 10, to both Ram and Shyam.

Ram computes 18 as secret using the forged key.
Shyam computes 11 as secret using the forged key.

Hari computes 18 as Ram's secret and 11 as Shyams's secret using the forged key.
The forged key leads Ram and Shyam to compute different shared secrets.
Still neither Ram nor Shyam is aware that they are communicating through Hari.
The core vulnerability lies in the lack of authentication.
Without authentication of the parties involved, Hari can easily impersonate one of the parties and establish separate keys with both.
This allows him to intercept and manipulate and tamper with the communications.
However, this mismatch in shared secrets still could cause errors in encryption and decryption, making Ram and Shyam suspect of an attack.

Hence, Interception by Hari MAY fail!
